In [17]:
import os.path
import sqlite3
import cohere
import numpy as np

In [18]:
def give_list(db, name=''):
	if len(name) != 0:
		#print(name + ':')
		res_table = db.execute(f'SELECT * FROM {name}')
		rows = [row[0] for row in res_table]
		return rows
	else:
		rows = [row[0] for row in db]
		return rows

In [19]:
title = "bruh, and muh v. bruh, sdfsd"
', '.join(title.split(', ')[:-1]).split('(')[0].rstrip()

'bruh, and muh v. bruh'

In [20]:
con = sqlite3.connect('courtCases_old.db')
cur = con.cursor()

In [21]:
command = """
SELECT name
FROM Cases C
"""

res_table = cur.execute(command)
names = give_list(res_table)

command = """
SELECT content
FROM Cases C
"""

res_table = cur.execute(command)
content = give_list(res_table)

docs = [n + ':\n' + c for n in names for c in content]

In [22]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [23]:
cohere_key = os.getenv("COHERE_API_KEY")
co = cohere.Client(cohere_key)

In [24]:
doc_emb = co.embed(docs, input_type="search_document", model="embed-english-v3.0").embeddings
doc_emb = np.asarray(doc_emb)
doc_emb

array([[ 0.05593872, -0.0021801 ,  0.01139831, ...,  0.02645874,
        -0.03637695,  0.01754761],
       [-0.00680542, -0.02107239, -0.0008564 , ...,  0.01207733,
        -0.02563477,  0.0178833 ],
       [ 0.04556274,  0.02868652,  0.01488495, ..., -0.02157593,
         0.02545166,  0.01864624],
       ...,
       [ 0.03811646,  0.00497818,  0.06744385, ...,  0.05541992,
         0.01400757,  0.00035119],
       [ 0.03265381, -0.03857422,  0.03231812, ..., -0.01438904,
        -0.01423645,  0.02304077],
       [ 0.02694702, -0.02528381,  0.03274536, ..., -0.03622437,
        -0.01242828,  0.01734924]])

In [25]:
doc_emb.shape

(40000, 1024)

In [26]:
#Encode your query with input type 'search_query'
query = "Is there any precedent for calling a second-hand witness to the stand?"
query_emb = co.embed([query], input_type="search_query", model="embed-english-v3.0").embeddings
query_emb = np.asarray(query_emb)
query_emb.shape

(1, 1024)

In [27]:
#Compute the dot product between query embedding and document embedding
scores = np.dot(query_emb, doc_emb.T)[0]

#Find the highest scores
max_idx = np.argsort(-scores)

count = 0
print(f"Query: {query}")
for idx in max_idx:
  print(f"Score: {scores[idx]:.2f}")
  print(docs[idx])
  print("--------")
  count += 1
  if count == 3:
    break;

Query: Is there any precedent for calling a second-hand witness to the stand?
Score: 0.44
ATCHISON: ORDER

This case is restored to the calendar for reargument. Counsel are requested to brief and discuss during oral argument the following questions: [434 U.S. 980 , 981]   1. Is the rule heretofore applied in the federal courts-that jeopardy attaches in jury trials when the jury is sworn-constitutionally mandated?

2. Should this Court hold that the Constitution does not require jeopardy to attach in any trial-state or federal, jury or nonjury- until the first witness is sworn?

The Solicitor General is invited to file a brief expressing th views of the United States on each of these questions.

Mr. Justice MARSHALL, dissenting.

By its order restoring this case to the calendar for rebriefing and additional oral argument, the Court appears once again to be "reach[ing] out" for a vehicle to change a long line of precedent. See Pennsylvania v. Mimms, 434 U.S. 106 , at 117, (STEVENS, J., d

In [28]:
def cosine_sim(vector, matrix):
    scores = []
    for mat in matrix:
        cos_sim = np.dot(vector, mat)/(np.linalg.norm(vector)*np.linalg.norm(mat))
        scores.append(cos_sim[0])
    return np.asarray(scores)

#Compute the cosine similarity between query embedding and document embedding
scores = cosine_sim(query_emb, doc_emb)

#Find the highest scores
max_idx = np.argsort(-scores)

count = 0
print(f"Query: {query}")
for idx in max_idx:
  print(f"Score: {scores[idx]:.2f}")
  print(docs[idx])
  print("--------")
  count += 1
  if count == 3:
    break;

Query: Is there any precedent for calling a second-hand witness to the stand?
Score: 0.44
ATCHISON: ORDER

This case is restored to the calendar for reargument. Counsel are requested to brief and discuss during oral argument the following questions: [434 U.S. 980 , 981]   1. Is the rule heretofore applied in the federal courts-that jeopardy attaches in jury trials when the jury is sworn-constitutionally mandated?

2. Should this Court hold that the Constitution does not require jeopardy to attach in any trial-state or federal, jury or nonjury- until the first witness is sworn?

The Solicitor General is invited to file a brief expressing th views of the United States on each of these questions.

Mr. Justice MARSHALL, dissenting.

By its order restoring this case to the calendar for rebriefing and additional oral argument, the Court appears once again to be "reach[ing] out" for a vehicle to change a long line of precedent. See Pennsylvania v. Mimms, 434 U.S. 106 , at 117, (STEVENS, J., d

In [29]:
max_idx

array([  241, 23441, 14841, ..., 21247,  9447, 14447])

In [31]:
results = co.rerank(query=query, documents=[docs[idx] for idx in max_idx][:50], top_n=3, model="rerank-english-v2.0")

print(f"Query: {query}")
for res in results:
  print(res.document['text'])
  print("--------")

Query: Is there any precedent for calling a second-hand witness to the stand?
WESTERN UNION TEL. CO. v. JAMES: ORDER

This case is restored to the calendar for reargument. Counsel are requested to brief and discuss during oral argument the following questions: [434 U.S. 980 , 981]   1. Is the rule heretofore applied in the federal courts-that jeopardy attaches in jury trials when the jury is sworn-constitutionally mandated?

2. Should this Court hold that the Constitution does not require jeopardy to attach in any trial-state or federal, jury or nonjury- until the first witness is sworn?

The Solicitor General is invited to file a brief expressing th views of the United States on each of these questions.

Mr. Justice MARSHALL, dissenting.

By its order restoring this case to the calendar for rebriefing and additional oral argument, the Court appears once again to be "reach[ing] out" for a vehicle to change a long line of precedent. See Pennsylvania v. Mimms, 434 U.S. 106 , at 117, (STE

In [ ]:
for i in range(len(docs)):
    if docs[i][:20].lower == results[0].document['text'][:20]:
        print(i)
        break;